In [1]:
import sys
import numpy as np
import numba
from sklearn.neighbors import NearestNeighbors

sys.path.append('../lib/')
from utils import *

### Test data

In [2]:
Nc = 1000
Ne = 2*Nc

c = 0.25*np.random.random(Nc)
sig = np.ones(Nc)*0.05
mu_center = np.random.random((Nc,2))
mu_eval = np.random.random((Ne,2))
xc = mu_center[:,0]
yc = mu_center[:,1]
xe = mu_eval[:,0]
ye = mu_eval[:,1]

### 1.- First _naive_ implementation

In [3]:
u1 = u_eval(c, sig, xc, yc, xe, ye)

In [4]:
tr1 = %timeit -o -n 100 u_eval(c, sig, xc, yc, xe, ye)

30.1 ms ± 492 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


***

### 2.- Second implementation: Gaussian Truncation

In [28]:
def compute_neighborhood(mu_center, mu_eval, maxsig):
    nn = NearestNeighbors(radius=maxsig, algorithm="ball_tree", n_jobs=2)
    nn.fit(mu_center)
    neigh_indexes_arr = nn.radius_neighbors(mu_eval, return_distance=False)
    neigh_indexes = []
    neigh_indexes_aux = []
    last = 0
    for arr in neigh_indexes_arr:
        l = arr.tolist(); l.sort()
        last += len(l)
        for index in l: neigh_indexes.append(index)
        neigh_indexes_aux.append(last)
    return np.asarray(neigh_indexes),np.asarray(neigh_indexes_aux)  

In [29]:
neigh_indexes,neigh_indexes_aux = compute_neighborhood(mu_center, mu_eval, 5*np.max(sig))

In [30]:
@numba.jit(nopython=True)
def u_eval_fast(c, sig, xc, yc, xe, ye, 
                neigh_indexes, neigh_indexes_aux):
    m = len(xe)
    n = len(xc)
    ret = np.zeros(m)
    sind = 0 # start index
    for i in range(m):
        eind = neigh_indexes_aux[i] # end index
        for j in neigh_indexes[sind:eind]:
            dist2 = (xe[i]-xc[j])**2 + (ye[i]-yc[j])**2
            ret[i] += c[j] * exp( -0.5 * dist2 / sig[j]**2 )
        sind = eind
    return ret

In [31]:
u2 = u_eval_fast(c,sig,xc,yc,xe,ye,neigh_indexes,neigh_indexes_aux)

In [10]:
tr2 = %timeit -o -n 100 u_eval_fast(c,sig,xc,yc,xe,ye,neigh_indexes,neigh_indexes_aux)

5.91 ms ± 337 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
print("Speedup: {0}".format(tr1.average/tr2.average))

Speedup: 5.101075328078544


In [12]:
print("Infinity norm of u1-u2: {0} ".format(np.max(np.abs(u1-u2))))

Infinity norm of u1-u2: 1.218100164290803e-05 


It can be further improved with bigger values of `maxsig`.

### 3.- _Naive_ + paralelization

In [13]:
from threading import Thread

In [14]:
@numba.jit(nopython=True, nogil=True)
def _u_eval_thread(c, sig, xc, yc, xe, ye, sind, chunk_size, res):
    n = len(xc)
    for i in range(sind,sind+chunk_size):
        for j in range(n):
            dist2 = (xe[i]-xc[j])**2 + (ye[i]-yc[j])**2
            res[i] += c[j] * exp( -0.5 * dist2 / sig[j]**2 )
            
def u_eval_thread(c, sig, xc, yc, xe, ye, n_thread=2):
    # array for results
    Ne = len(xe)
    res = np.zeros(Ne)
    
    # data size for each thread
    chunk_size = Ne // n_thread
    
    # starting index for each thread
    start_indexes = [i * chunk_size for i in range(n_thread)]
    
    threads = [Thread(target=_u_eval_thread, 
                  args=(c,sig,xc,yc,xe,ye,sind,chunk_size,res)) for sind in start_indexes]
    for thread in threads: thread.start()
    for thread in threads: thread.join()
        
    return res

In [15]:
u3 = u_eval_thread(c, sig, xc, yc, xe, ye, n_thread=2)

In [16]:
tr3 = %timeit -o -n 100 u_eval_thread(c, sig, xc, yc, xe, ye, n_thread=2)

16.5 ms ± 505 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
print("Speedup: {0}".format(tr1.average/tr3.average))

Speedup: 1.828799099791643


In [18]:
print("Infinity norm of u1-u3: {0} ".format(np.max(np.abs(u1-u3))))

Infinity norm of u1-u3: 0.0 


### 4.- Gaussian truncation + paralelization

In [19]:
@numba.jit(nopython=True)
def _u_eval_fast(c, sig, xc, yc, xe, ye, neigh_indexes, 
                 neigh_indexes_aux, _sind, chunk_size, ret):
    sind = 0 # start index
    if _sind!=0: sind = neigh_indexes_aux[_sind-1]
    for i in range(_sind,_sind+chunk_size):
        eind = neigh_indexes_aux[i] # end index
        for j in neigh_indexes[sind:eind]:
            dist2 = (xe[i]-xc[j])**2 + (ye[i]-yc[j])**2
            ret[i] += c[j] * exp( -0.5 * dist2 / sig[j]**2 )
        sind = eind


def u_eval_thread_fast(c, sig, xc, yc, xe, ye, neigh_indexes,
                       neigh_indexes_aux, n_thread=2):
    # array for results
    Ne = len(xe)
    ret = np.zeros(Ne)
    
    # data size for each thread
    chunk_size = Ne // n_thread
    
    # starting index for each thread
    start_indexes = [i * chunk_size for i in range(n_thread)]
    
    threads = [Thread(target=_u_eval_fast, 
                  args=(c,sig,xc,yc,xe,ye,neigh_indexes,neigh_indexes_aux,sind,chunk_size,ret)) for sind in start_indexes]
    for thread in threads: thread.start()
    for thread in threads: thread.join()
        
    return ret

In [20]:
u4 = u_eval_thread_fast(c, sig, xc, yc, xe, ye, neigh_indexes, neigh_indexes_aux, n_thread=2)

In [21]:
tr4 = %timeit -o -n 100 u_eval_thread_fast(c, sig, xc, yc, xe, ye, neigh_indexes, neigh_indexes_aux, n_thread=2)

6.61 ms ± 170 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
print("Speedup: {0}".format(tr1.average/tr4.average))

Speedup: 4.5592654835121165


In [23]:
print("Infinity norm of u1-u4: {0} ".format(np.max(np.abs(u1-u4))))

Infinity norm of u1-u4: 1.218100164290803e-05 


### 5.- figtree

In [24]:
from pyfigtree import figtree

In [25]:
# we set epsilon to match the error bound of the Gaussian truncation approach
u5 = figtree(mu_center, mu_eval, c, bandwidth=0.05, epsilon=1e-6)

In [26]:
%timeit -n 100 figtree(mu_center, mu_eval, c, bandwidth=0.05, epsilon=1e-5)

11.1 ms ± 501 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [27]:
print("Infinity norm of u1-u5: {0} ".format(np.max(np.abs(u1-u5))))

Infinity norm of u1-u5: 1.4866045517363546 


## Time comparison

In [32]:
times1 = []
times2 = []
times3 = []
times4 = []
n_points = list(range(250, 5001, 250))

for Nc in n_points:
    # points generation
    Ne = 2*Nc
    mu_center = np.random.random((Nc,2))
    mu_eval = np.random.random((Ne,2))
    c = 0.25*np.random.random(Nc)
    sig = np.ones(Nc)*0.05
    xc = mu_center[:,0]
    yc = mu_center[:,1]
    xe = mu_eval[:,0]
    ye = mu_eval[:,1]
    neigh_indexes,neigh_indexes_aux = compute_neighborhood(mu_center, mu_eval, 5*np.max(sig))
    
    # evaluating performance of direct evaluation (naive)
    tr1 = %timeit -o -n 3 -q u_eval(c, sig, xc, yc, xe, ye)
    times1.append(tr1.average)
    
    # evaluating performance of Gaussian truncation
    tr2 = %timeit -o -n 3 -q u_eval_fast(c,sig,xc,yc,xe,ye,neigh_indexes,neigh_indexes_aux)
    times2.append(tr2.average)
    
    # evaluating performance of Naive + paralelization
    tr3 = %timeit -o -n 3 -q u_eval_thread(c, sig, xc, yc, xe, ye, n_thread=2)
    times3.append(tr3.average)
    
    # evaluating performance of Gaussian truncation + paralelization
    tr4 = %timeit -o -n 3 -q u_eval_thread_fast(c, sig, xc, yc, xe, ye, neigh_indexes, neigh_indexes_aux, n_thread=2)
    times4.append(tr4.average)

KeyboardInterrupt: 